In [ ]:
# import libraries
try:
  # Uninstall conflicting packages for a clean install
  !pip uninstall -y tensorflow tf-nightly absl-py
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
# 1. Load data into Pandas DataFrames
train_df = pd.read_csv(train_file_path, sep='	', names=["label", "message"])
test_df = pd.read_csv(test_file_path, sep='	', names=["label", "message"])

# 2. Preprocess data
# Convert labels to numerical values: ham=0, spam=1
train_df["label"] = train_df["label"].map({"ham": 0, "spam": 1})
test_df["label"] = test_df["label"].map({"ham": 0, "spam": 1})

# Separate text and labels
train_labels = train_df.pop("label")
train_text = train_df.pop("message")
test_labels = test_df.pop("label")
test_text = test_df.pop("message")

# 3. Text Vectorization
# Define the vectorizer layer to convert text to integer sequences
VOCAB_SIZE = 10000
MAX_SEQUENCE_LENGTH = 250

vectorizer = tf.keras.layers.TextVectorization(
    max_tokens=VOCAB_SIZE,
    output_sequence_length=MAX_SEQUENCE_LENGTH,
    output_mode='int'
)

# Adapt the vectorizer to the training text to build the vocabulary
vectorizer.adapt(train_text.values)

# 4. Build the Model
model = tf.keras.Sequential([
    # Input layer expects raw strings
    tf.keras.Input(shape=(1,), dtype=tf.string),

    # Vectorization layer
    vectorizer,

    # Embedding layer: Transforms integers to dense vectors
    tf.keras.layers.Embedding(len(vectorizer.get_vocabulary()), 32),

    # LSTM layer for sequence processing
    tf.keras.layers.LSTM(32),

    # Dense layers for classification
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid') # Output 0-1 probability
])

# 5. Compile and Train
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

# Train the model
history = model.fit(
    train_text.values,
    train_labels,
    epochs=10,
    validation_data=(test_text.values, test_labels),
    verbose=2
)

# 6. Prediction Function
def predict_message(pred_text):
    # Model expects a list of strings, so we wrap the input text in a list
    prediction = model.predict([pred_text])

    # Extract the probability value (float)
    p = prediction[0][0]

    # Determine label based on probability threshold
    if p < 0.5:
        label = "ham"
    else:
        label = "spam"

    return [p, label]

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  # Model expects a string input, so convert the input text to a TensorFlow string tensor
  input_tensor = tf.constant([pred_text], dtype=tf.string)
  prediction = model.predict(input_tensor)

  # Extract the probability value (float)
  p = prediction[0][0]

  # Determine label based on probability threshold
  if p < 0.5:
      label = "ham"
  else:
      label = "spam"

  return [p, label]

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
